# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key


In [15]:
geoapify_key

'3d162d056bd34198944194464cd123d7'

In [16]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,polyarnyy,69.1989,33.4478,277.24,66,82,9.02,RU,1728840115
1,1,kharkhorin,47.1975,102.8238,271.17,86,99,3.03,MN,1728840117
2,2,stanley,54.8680,-1.6985,279.55,72,0,0.51,GB,1728840118
3,3,nurota,40.5614,65.6886,283.94,81,23,0.56,UZ,1728840050
4,4,edinburgh of the seven seas,-37.0676,-12.3116,287.03,75,90,9.67,SH,1728840120


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [17]:
# Check the actual column names in the DataFrame
print(city_data_df.columns)


Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [18]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(
    'Lng',  
    'Lat',  
    size='Humidity',  
    geo=True,  
    color='blue',  
    tiles='OSM',  
    frame_width=900,  
    frame_height=500  
)

# Display the map
city_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [19]:
# Check the column names in city_data_df
print(city_data_df.columns)

Index(['City_ID', 'City', 'Lat', 'Lng', 'Max Temp', 'Humidity', 'Cloudiness',
       'Wind Speed', 'Country', 'Date'],
      dtype='object')


In [20]:
# Narrow down cities that fit criteria and drop any results with null values
# Narrow down cities that fit the ideal weather conditions
ideal_weather_df = city_data_df.loc[(city_data_df["Max Temp"]>21) & (city_data_df["Max Temp"]< 27) &
                             (city_data_df["Wind Speed"]< 4.5) & (city_data_df["Cloudiness"] == 0) ]
ideal_weather_df

# Drop any rows with null values
ideal_weather_df.dropna(how="any")

# Display the filtered data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
print(hotel_df.head())  # Check the first few rows
print(hotel_df.shape)    # Check the shape to see if it has rows and columns

Empty DataFrame
Columns: [City, Country, Lat, Lng, Hotel Name]
Index: []
(0, 5)


In [22]:
hotel_df

,City,Country,Lat,Lng,Hotel Name


In [24]:
hotel_df.reset_index(drop=True, inplace=True)

In [25]:
import requests

# Set the search radius to 10,000 meters (10 km)
radius = 10000

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude and longitude from the DataFrame for each city
    lat = row['Lat']
    lng = row['Lng']

    # Add the current city's latitude and longitude to the params dictionary
    params = {
        "categories": "accommodation.hotel",  # Search for hotels
        "filter": f"circle:{lng},{lat},{radius}",  # Filter based on city coordinates and radius
        "bias": f"proximity:{lng},{lat}",  # Bias the search towards the city's location
        "limit": 1,  # Limit results to the first hotel
        "apiKey": geoapify_key  # Your Geoapify API key
    }

    # Set base URL for the Geoapify API
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = response.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df.head()

Starting hotel search


,City,Country,Lat,Lng,Hotel Name


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [26]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    'Lng', 'Lat',  
    geo=True,      
    size=10,       
    color='Hotel Name',  
    tiles='OSM',   
    hover_cols=['City', 'Country', 'Hotel Name'],  
    frame_width=800, 
    frame_height=600
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (Hotel Name,City,Country)